In [ ]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 26.6 MB/s eta 0:00:00


In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.6 MB/s eta 0:00:00


In [ ]:
import fitz  # PyMuPDF
import PyPDF2
import io

def highlight_repeated_words(input_pdf_path, output_pdf_path):
    pdf_document = fitz.open(input_pdf_path)

    # Extract text from the PDF using PyMuPDF
    pdf_text = ""
    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num)
        pdf_text += page.get_text()

    # Find repeated words using a simple regex pattern
    pattern = r'\b(\w+)\s+\1\b'
    repeated_word_regex = re.compile(pattern, re.IGNORECASE)
    repeated_words = set(repeated_word_regex.findall(pdf_text))

    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num)

        for word in repeated_words:
            instances = re.finditer(r'\b{}\b'.format(re.escape(word)), page.get_text())
            for instance in instances:
                quad = page.rect_for_bbox(fitz.Rect(instance.start, instance.end))
                annot = page.add_highlight_annot(quad)
                annot.set_colors({"stroke": (1, 1, 0), "fill": (1, 1, 0)})  # Yellow highlighting

    pdf_document.save(output_pdf_path)
    pdf_document.close()

# Replace 'input.pdf' and 'output.pdf' with your file paths
input_pdf_file = '/content/Inputs.pdf'
output_pdf_file = 'output.pdf'

highlight_repeated_words(input_pdf_file, output_pdf_file)

In [ ]:
import fitz  # PyMuPDF
import PyPDF2
import io

def highlight_repeated_words(input_pdf_path, output_pdf_path):
    pdf_document = fitz.open(input_pdf_path)

    # Extract text from the PDF using PyMuPDF
    pdf_text = ""
    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num)
        pdf_text += page.get_text()

    # Find repeated words using a simple regex pattern
    pattern = r'\b(\w+)\s+\1\b'
    repeated_word_regex = re.compile(pattern, re.IGNORECASE)
    repeated_words = set(repeated_word_regex.findall(pdf_text))

    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num)

        for word in repeated_words:
            instances = re.finditer(r'\b{}\b'.format(re.escape(word)), page.get_text())
            for instance in instances:
                quad = page.rect_for_bbox(fitz.Rect(instance.start, instance.end))
                annot = page.add_highlight_annot(quad)
                annot.set_colors({"stroke": (1, 1, 0), "fill": (1, 1, 0)})  # Yellow highlighting

    pdf_document.save(output_pdf_path)
    pdf_document.close()

# Replace 'input.pdf' and 'output.pdf' with your file paths
input_pdf_file = '/content/Inputs.pdf'
output_pdf_file = 'output.pdf'

highlight_repeated_words(input_pdf_file, output_pdf_file)

In [ ]:
import fitz  # PyMuPDF

def highlight_word_in_pdf(input_pdf_path, output_pdf_path, word_to_highlight):
    pdf_document = fitz.open(input_pdf_path)

    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num)
        text_instances = page.search_for(word_to_highlight)

        for inst in text_instances:
            highlight = page.add_highlight_annot(inst)
            highlight.set_colors({"stroke": (1, 1, 0), "fill": (1, 1, 0)})  # Yellow highlighting

    pdf_document.save(output_pdf_path)
    pdf_document.close()

# Replace 'input.pdf' and 'output.pdf' with your file paths
input_pdf_file = '/content/Inputs.pdf'
output_pdf_file = 'output.pdf'

# Highlight the word "Pathology" in the PDF
highlight_word_in_pdf(input_pdf_file, output_pdf_file, "Pathology")


In [ ]:
!pip install transformers pymupdf

In [ ]:
import fitz  # PyMuPDF for PDF handling
from transformers import BertTokenizer, BertForTokenClassification
import torch

# Load BioBERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
model = BertForTokenClassification.from_pretrained("dmis-lab/biobert-base-cased-v1.1")

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as pdf:
        for page in pdf:
            text += page.get_text()
    return text

# Function for Named Entity Recognition (NER) using BioBERT
def perform_ner(text):
    inputs = tokenizer.encode_plus(text, return_tensors="pt", add_special_tokens=True)
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=2)

    # Decode the predictions
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    entities = []
    entity = ""
    inside_entity = False
    for token, prediction in zip(tokens, predictions[0].numpy()):
        if token.startswith("##"):
            entity += token[2:]
        else:
            entity += " " + token

        if inside_entity:
            if not token.startswith("##"):
                entities.append(entity.strip())
                inside_entity = False
                entity = ""
        else:
            if token.startswith("B-"):
                inside_entity = True

    return entities

# Function to identify repeated entities
def identify_repetitions(entities):
    repetitions = {}
    for entity in entities:
        if entity in repetitions:
            repetitions[entity] += 1
        else:
            repetitions[entity] = 1
    return repetitions

# PDF path
pdf_path = 'Inputs.pdf'

# Extract text from PDF
pdf_text = extract_text_from_pdf(pdf_path)

# Perform Named Entity Recognition (NER) using BioBERT
recognized_entities = perform_ner(pdf_text)

# Identify repetitions of recognized entities
repeated_entities = identify_repetitions(recognized_entities)

print("Repeated Entities:")
print(repeated_entities)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Repeated Entities:
{}


In [ ]:
from transformers import BertTokenizer, BertForTokenClassification
import torch

# Load the tokenizer and model (replace with a suitable biomedical model from Hugging Face)
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
model = BertForTokenClassification.from_pretrained("bert-base-cased", num_labels=2)  # Example: 2 labels including 'MEDICINE'

# Sample text (replace this with your text)
sample_text = "The patient was prescribed paracetamol,paracetamol,ibuprofen and ibuprofen."

# Tokenize the text
tokens = tokenizer.encode_plus(
    sample_text,
    add_special_tokens=True,
    return_token_type_ids=False,
    return_attention_mask=True,
    return_tensors="pt"
)

# Perform Named Entity Recognition (NER)
with torch.no_grad():
    outputs = model(**tokens)
    predictions = torch.argmax(outputs.logits, dim=2)

# Convert token ids back to tokens
predicted_tokens = [tokenizer.decode(token) for token in predictions[0]]

# Identify repeated mentions of medicines
medicines = []
previous_token = None
for token, pred_label in zip(predicted_tokens, predictions[0]):
    # Assuming label 1 corresponds to medicine entity in the model's output
    if pred_label.item() == 1:
        if token == previous_token:
            medicines.append(token)
        previous_token = token
    else:
        previous_token = None

# Print the repeated medicines
if medicines:
    print("Repeated medicines:", medicines)
else:
    print("No repeated medicines found.")

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Repeated medicines: ['[ u n u s e d 1 ]', '[ u n u s e d 1 ]', '[ u n u s e d 1 ]', '[ u n u s e d 1 ]', '[ u n u s e d 1 ]']


In [ ]:
from transformers import BertTokenizer, BertForTokenClassification
import torch

# Load the tokenizer and model (replace with a suitable biomedical model from Hugging Face)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=2)  # Example: 2 labels including 'MEDICINE'

# Sample text with repetition of medicines (customize this text as needed)
sample_text = "The patient was prescribed aspirin and ibuprofen. Ibuprofen was administered again for pain relief. The doctor suggested continuing with aspirin for the next few days."

# Tokenize the text
tokens = tokenizer.encode_plus(
    sample_text,
    add_special_tokens=True,
    return_token_type_ids=False,
    return_attention_mask=True,
    return_tensors="pt"
)

# Perform Named Entity Recognition (NER)
with torch.no_grad():
    outputs = model(**tokens)
    predictions = torch.argmax(outputs.logits, dim=2)

# Convert token ids back to tokens
predicted_tokens = [tokenizer.decode(token) for token in predictions[0]]

# Identify repeated mentions of medicines
medicines = []
previous_token = None
for token, pred_label in zip(predicted_tokens, predictions[0]):
    # Assuming label 1 corresponds to medicine entity in the model's output
    if pred_label.item() == 1:
        if token == previous_token:
            medicines.append(token)
        previous_token = token
    else:
        previous_token = None

# Print the repeated medicines
if medicines:
    print("Repeated medicines:", medicines)
else:
    print("No repeated medicines found.")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Repeated medicines: ['[ u n u s e d 0 ]', '[ u n u s e d 0 ]', '[ u n u s e d 0 ]', '[ u n u s e d 0 ]', '[ u n u s e d 0 ]', '[ u n u s e d 0 ]', '[ u n u s e d 0 ]', '[ u n u s e d 0 ]', '[ u n u s e d 0 ]']


In [ ]:
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_sm-0.4.0.tar.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 19.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 9.3 MB/s eta 0:00:00
  Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
  Created wheel for nmslib: filename=nmslib-2.1.1-cp310-cp310-linux_x86_64.whl size=13578645 sha256=2beaaa326b325cfb1b7d58f4081c149275ebf4a20c6e53838087f0c356934f50
  Stored in directory: /root/.cache/pip/wheels/21/1a/5d/4cc754a5b1a88405cad184b76f823897a63a8d19afcd4b9314
Successfully built nmslib
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully uninstalled scipy-1.11.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is t

In [ ]:
import spacy
from collections import Counter

# Load the scispaCy model for biomedical NER
nlp = spacy.load("en_core_sci_sm")

# Sample text with medicine mentions
sample_text = ("The patient was prescribed paracetamol and ibuprofen. Ibuprofen was administered again for pain relief. The doctor suggested continuing with paracetamol for the next few days."
)


# Process the text using the model
doc = nlp(sample_text)

# Extract medicine entities
medicine_entities = [ent.text.lower() for ent in doc.ents if ent.label_ == "CHEMICAL"]

# Identify repetitions
repeated_medicines = [medicine for medicine, count in Counter(medicine_entities).items() if count > 1]

# Print repeated medicines
if repeated_medicines:
    print("Repeated medicines:", repeated_medicines)
else:
    print("No repeated medicines found.")

No repeated medicines found.


In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.4 MB/s eta 0:00:00


In [ ]:
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PdfReader(file)
        text = ''
        for page_num in range(len(reader.pages)):
            text += reader.pages[page_num].extract_text()
    return text

# Example usage:
pdf_file_path = '/content/Inputs (1).pdf'  # Update this with your PDF file path
extracted_text = extract_text_from_pdf(pdf_file_path)
print(extracted_text)

%, T. i 4 PI A L Gilat Bazar, Varanasi - 221002 (India) 
. Phone - + 91 ~ 542 - 2284999, Emergency Tel -+ 91 88743 22226 
4 promise for healthy | ife Email -info@trimurtthospital.com, Web - www.trimurtihospital.com 
An (SO certified hospital. 
Sima oe 
Admission Date 02-Aug-2021 
UID $ Age 45 Year Male Admission Time — :03:04:53PM. 
TED No. —_ Discharge Date :07-Aug-2021 
Patient Name + Discharge Time = :05:43:17PM 
Doctor Name? + Dh R si MURTI SINGH Company Name; 
PRO. Services name Amount 
3 i Réom/ Bed Charves: RS 350086 DAY 21,000 60 
| > | Nursing Charges 500*6 DAY 3,000.00 
i Consultant Charges 1000%6 DAY 6,000.00 
1) 4 | Blood Sugar o0"10 900.00 
S ECG 300°! 300.00 
6 | Medicine 19,397.00 
‘7 | INVESTIGATION DETAILS 
Pathology 2,840.00 
Pathology 4,560.00 
Ultrasound 1,000.00 
X-Ray 700,00 
| | i | 
| 
a eer Total reget 
Discount: 0,00 
Less Advance: 0.00 
Net Amount; $9,697.00 
Paid Amount: 0.00 
Balance ; 59,697.00 
Rs.Fifty Nine Thousand Six Hundred Ninty Seven Only 
Reccipen

In [ ]:
!pip install azure-ai-formrecognizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.1/300.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient

# Set your Azure Form Recognizer API key and endpoint
api_key = "3ad96d94817341bd897a582d4157b6cf"
endpoint = "https://fraud-azure.cognitiveservices.azure.com/"

# Create Azure credentials
credential = AzureKeyCredential(api_key)

# Create a Document Analysis client
doc_analysis_client = DocumentAnalysisClient(endpoint, credential)

# Path to the PDF document you want to analyze
pdf_path = "/content/output1.pdf"

# Read the PDF file content
with open(pdf_path, "rb") as f:
    pdf_data = f.read()

# Submit the document for analysis
poller = doc_analysis_client.begin_analyze_document("prebuilt-layout", pdf_data)
result = poller.result()

# Process the analysis result
for page in result.pages:
    print("Page number: {}".format(page.page_number))
    for line in page.lines:
        print(line.content)

Page number: 1
......
.....
TRIMURTI
HOSPITAL
A promise for healthy life
Gilat Bazar, Varanasi - 221002 (India)
Phone - + 91 - 542 - 2284999, Emergency Tel - + 91 88743 22226
Email - info@trimurtihospital.com, Web - www.trimurtihospital.com
An ISO certified hospital.
Final Bill
BỊIl NO
UHID
IED No.
Patient Name :
Doctor Name :
Dr. RAM MURTI SINGH
Bed No
:
PVT-21
Age :45 Year
Male
Admission Date
:02-Aug-2021
Admission Time
:03:04:53PM
Discharge Date
:07-Aug-2021
Discharge Time
:05:43:17PM
Company Name
:
Srno.
Services name
Qty.
Amount
Room/Bed Charges
RS 3500*6 DAY
21,000.00
2
Nursing Charges
500*6 DAY
3.000.00
Consultant Charges
1000% DAY
6,000.00
4
Blood Sugar
90*10
900.00
5
ECG
300*1
300.00
6
Medicine
19,397.00
7
INVESTIGATION DETAILS
Pathology
2,840.00
Pathology
4,560.00
Ultrasound
1,000.00
X-Ray
700.00
Total Charge :
59,697.00
Discount:
0,00
0.00
Less Advance:
Net Amount :
59,697.00
Paid Amount:
0.00
Balance :
59.697.00
Rs.Fifty Nine Thousand Six Hundred Ninty Seven Only
ardlac Car

In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.9 MB/s eta 0:00:00


In [ ]:
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        pdf_reader = PdfReader(file)
        text = ""
        for page_number in range(len(pdf_reader.pages)):
            text += pdf_reader.pages[page_number].extract_text()
    return text

# Replace 'path/to/your.pdf' with your actual PDF file path
pdf_path = '/content/NHI data requirement #2 (ins fraud detection).pdf'
extracted_text = extract_text_from_pdf(pdf_path)
print(extracted_text)

SUSHRUTHA MEDICAL CENTRE Pothencode, Thiruvananthapuram ‘ Ph:0471 3247745,419937 Mob:9495989937 
Patient Charge Details 
Registration # ; geese 
Bill # : = Name | See, 
Bill Date: 18/05/2018 Customer * ecient esses pesereco 
Ward : SINGLE ROOM Address 1 baba jerteiaeest ay 
Bed# =: A2 
Si# Date Bill # Item Name 
Qty Amount 1 ANAESTHETIST FEE 
1 13/05/2018 SUN32 Icu 1 6000.00 
Sub Total : 6000.00 1 OT CHARGE 
1 13/05/2018 SUN32 ICU 
1 10000 00 
Sub Total : 10000.00 1 OT CONSUMABLES 
1 13/05/2018 sun32 IcU 
1 2000.00 
Sub Total ; 2000.00 1 OT PROF.ASSI. CHARGES 
1 13/05/2018 SUN32 ICU 
1 $000.00 
Sub Total : 8000.00 ANAESETHESIA ASSISTANT 
1 13/05/2018 — SyNn32 ICU 1 1000.00 
Sub Total : 1000.00 ASSISTANT FEE 
1 13/05/2018 — SUN32 Icu re 1 3500.00 
. Sub Total t 3500.00 BED CHARGE 
i 13/05/2018 To 14/05/2018 08:: BED CHARGE Bed# ADL POPS (20 Hours) 1 350.00 
Sub Total : 350.00 BIOMEDICAL WASTE CHARGE ee 
1 11/05/2018 BIOMEDICAL WASTE CHARGE Bed# A2 (15 Hours) 25.00 2 12/05/2018 BIOMEDICA

In [ ]:
!pip install azure-ai-textanalytics

In [ ]:
# Replace with your Azure Text Analytics endpoint and key
endpoint = "https://eagle-eyes.cognitiveservices.azure.com/"
key = "de399cd9565a44028fbce14826266cc5"

from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

# Authenticate the client using your key and endpoint
def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint,
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()




# Example function for extracting information from healthcare-related text
def health_example(client):
    documents = [
        """
MEDICA AL CER SUSHRUTHA MEDICAL CENTRE SO POTHENCODE OF D Pothencode, Thiruvananthapuram Ph:04713247745,419937 Mob:9495989937 BOSUSRUTH Patient Charge Details Registration # Bill # Name Customer Bill Date : 18/05/2018 Address Ward : SINGLE ROOM Bed # : A2 SI # Date Bill # Item Name 1 ANAESTHETIST FEE Qty Amount 1 13/05/2018 SUN32 ICU 6000.00 Sub Total 1 OT CHARGE 6000.00 1 13/05/2018 SUN32 ICU 1 10000 00 Sub Total 1 OT CONSUMABLES 10000.00 1 13/05/2018 SUN32 ICU 1 2000.00 Sub Total 2000.00 1 OT PROF.ASSI. CHARGES 1 13/05/2018 SUN32 ICU 1 S000.00 Sub Total 8000.00 ANAESETHESIA ASSISTANT 1 13/05/2018 SUN32 ICU 1 1000.00 Sub Total 1000.00 ASSISTANT FEE 1 13/05/2018 SUN32 ICU 1 3500.00 Sub Total 3500.00 BED CHARGE 1 13/05/2018 To 14/05/2018 08 :: BED CHARGE Bed# ADL POPS (20 Hours) 1 350.00 Sub Total 350.00 BIOMEDICAL WASTE CHARGE 1 11/05/2018 BIOMEDICAL WASTE CHARGE Bed# A2 (15 Hours) 25.00 2 12/05/2018 BIOMEDICAL WASTE CHARGE Bed# A2 (54 Hours) 50.00 3 12/05/2018 BIOMEDICAL WASTE CHARGE Bed# ICU 1 (34 Hours) 20.00 4 13/05/2018 BIOMEDICAL WASTE CHARGE Bed# ADL POP5 (20 Hours) 15.00 5 14/05/2018 BIOMEDICAL WASTE CHARGE Bed# A2 (98 Hours) 125.00 Sub Total 235.00 CARDIAC MONITORING 1 16/05/2018 SVN192 CARDIAC MONITOR (PER HOUR) 3 150.00 Sub Total 150.00 CONSULTATION CHARGE 1 11/05/2018 CONSULTATION CHARGE Bed# A2 (15 Hours) 450.00 Page 1 of 6 Patient Charge Details registration # Bill # Name Bill Date : 18/05/2018 Customer Ward : SINGLE ROOM Address Bed # : A2 SI # Date Bill # Item Name Qty Amount 2 12/05/2018 CONSULTATION CHARGE Bed# ICU 1 (34 Hours) 600.00 3 13/05/2018 CONSULTATION CHARGE Bed# ADL POPS (20 Hours) 450.00 4 14/05/2018 CONSULTATION CHARGE Bed# A2 (98 Hours) 2250.00 Sub Total 3750.00 CONSUMABLES SALES 1 11/05/2018 19 DISPOSSIBLES A1 1 15.00 2 17/05/2018 20 DISPOSSIBLES A1 3 45.00 3 18/05/2018 2 DISPOSSIBLES A1 1 15.00 Sub Total 75.00 ICU CHARGES 1 12/05/2018 ICU CHARGES Bed# ICU 1 (34 Hours) 2700.00 Sub Total 2700.00 LABORATORY 1 11/05/2018 19 APTT 1 200.00 2 11/05/2018 19 POE 1 1860.00 3 16/05/2018 SVN190 GRBS 1 40.00 4 16/05/2018 SVN192 BIOPSY L 1 1000.00 5 16/05/2018 SVN192 DRAIN REMOVAL 1 50.00 6 17/05/2018 20 HBAIC 300.00 7 17/05/2018 20 RANDOM BLOOD SUGAR 3 45.00 8 18/05/2018 21 PCV 1 30.00 9 18/05/2018 21 HEAMOGLOBIN 1 30.00 10 18/05/2018 21 RANDOM BLOOD SUGAR 1 15.00 Sub Total 3570.00 MEDICINE CHARGES 1 11/05/2018 OPH1343 CEFBACT.S 1.5 2 309.99 2 11/05/2018 OPH1343 IVF DNS 500 ML INJ 2 62.38 3 11/05/2018 OPH1343 CANULA FIXATOR 1 45.00 4 11/05/2018 OPH1343 DISPOVAN 2 ML 1 4.50 5 11/05/2018 OPH1343 ENEMA PRACTO CLISYS 1 45.00 6 11/05/2018 OPH1343 GLOVES DISPO 1 58.00 7 11/05/2018 OPH1343 IV CANULA 18 G POLYMED 1 102.50 8 11/05/2018 OPH1343 IV SET POLYMED 1 80.00 9 11/05/2018 OPH1343 IVF N S 500 ML INJ 1 28.47 10 11/05/2018 OPH1343 SPIRIT SWAB 2 16.00 11 11/05/2018 OPH1343 IVF DEXTROSE 5 % 500 ML INJ 2 62.03 12 11/05/2018 OPH1343 DISPOVAN 10 ML 2 17.00 13 12/05/2018 OPH1401 ORNIDA 100 ML IV 3 149.99
        """
    ]

    poller = client.begin_analyze_healthcare_entities(documents)
    result = poller.result()

    docs = [doc for doc in result if not doc.is_error]

    for idx, doc in enumerate(docs):
        for entity in doc.entities:
            print("Entity: {}".format(entity.text))
            print("Category: {}".format(entity.category))

health_example(client)



Entity: 18/05/2018
Category: Date
Entity: 13/05/2018 SUN32
Category: Time
Entity: ICU
Category: CareEnvironment
Entity: 6000.00
Category: MeasurementValue
Entity: CHARGE
Category: AdministrativeEvent
Entity: 6000.00
Category: MeasurementValue
Entity: 13/05/2018 SUN32
Category: Time
Entity: ICU
Category: CareEnvironment
Entity: 13/05/2018 SUN32
Category: Time
Entity: ICU
Category: CareEnvironment
Entity: CHARGES
Category: AdministrativeEvent
Entity: 13/05/2018 SUN32
Category: Time
Entity: ICU
Category: CareEnvironment
Entity: ANAESETHESIA
Category: TreatmentName
Entity: 13/05/2018 SUN32
Category: Time
Entity: ICU
Category: CareEnvironment
Entity: 13/05/2018 SUN32
Category: Time
Entity: ICU
Category: CareEnvironment
Entity: CHARGE
Category: AdministrativeEvent
Entity: 13/05/2018
Category: Time
Entity: 14/05/2018
Category: Date
Entity: 08
Category: Time
Entity: CHARGE
Category: AdministrativeEvent
Entity: 20 Hours
Category: Time
Entity: 350.00
Category: MeasurementValue
Entity: BIOMEDICAL

In [ ]:
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

# Replace with your Azure Text Analytics endpoint and key
endpoint = "https://eagle-eyes.cognitiveservices.azure.com/"
key = "de399cd9565a44028fbce14826266cc5"

# Authenticate the client using your key and endpoint
def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint,
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()

# Function to extract medication names and dates from healthcare-related text
def extract_medication_and_dates(client):
    documents = [
        """
MEDICA AL CER SUSHRUTHA MEDICAL CENTRE SO POTHENCODE OF D Pothencode, Thiruvananthapuram Ph:04713247745,419937 Mob:9495989937 BOSUSRUTH Patient Charge Details Registration # Bill # Name Customer Bill Date : 18/05/2018 Address Ward : SINGLE ROOM Bed # : A2 SI # Date Bill # Item Name 1 ANAESTHETIST FEE Qty Amount 1 13/05/2018 SUN32 ICU 6000.00 Sub Total 1 OT CHARGE 6000.00 1 13/05/2018 SUN32 ICU 1 10000 00 Sub Total 1 OT CONSUMABLES 10000.00 1 13/05/2018 SUN32 ICU 1 2000.00 Sub Total 2000.00 1 OT PROF.ASSI. CHARGES 1 13/05/2018 SUN32 ICU 1 S000.00 Sub Total 8000.00 ANAESETHESIA ASSISTANT 1 13/05/2018 SUN32 ICU 1 1000.00 Sub Total 1000.00 ASSISTANT FEE 1 13/05/2018 SUN32 ICU 1 3500.00 Sub Total 3500.00 BED CHARGE 1 13/05/2018 To 14/05/2018 08 :: BED CHARGE Bed# ADL POPS (20 Hours) 1 350.00 Sub Total 350.00 BIOMEDICAL WASTE CHARGE 1 11/05/2018 BIOMEDICAL WASTE CHARGE Bed# A2 (15 Hours) 25.00 2 12/05/2018 BIOMEDICAL WASTE CHARGE Bed# A2 (54 Hours) 50.00 3 12/05/2018 BIOMEDICAL WASTE CHARGE Bed# ICU 1 (34 Hours) 20.00 4 13/05/2018 BIOMEDICAL WASTE CHARGE Bed# ADL POP5 (20 Hours) 15.00 5 14/05/2018 BIOMEDICAL WASTE CHARGE Bed# A2 (98 Hours) 125.00 Sub Total 235.00 CARDIAC MONITORING 1 16/05/2018 SVN192 CARDIAC MONITOR (PER HOUR) 3 150.00 Sub Total 150.00 CONSULTATION CHARGE 1 11/05/2018 CONSULTATION CHARGE Bed# A2 (15 Hours) 450.00 Page 1 of 6 Patient Charge Details registration # Bill # Name Bill Date : 18/05/2018 Customer Ward : SINGLE ROOM Address Bed # : A2 SI # Date Bill # Item Name Qty Amount 2 12/05/2018 CONSULTATION CHARGE Bed# ICU 1 (34 Hours) 600.00 3 13/05/2018 CONSULTATION CHARGE Bed# ADL POPS (20 Hours) 450.00 4 14/05/2018 CONSULTATION CHARGE Bed# A2 (98 Hours) 2250.00 Sub Total 3750.00 CONSUMABLES SALES 1 11/05/2018 19 DISPOSSIBLES A1 1 15.00 2 17/05/2018 20 DISPOSSIBLES A1 3 45.00 3 18/05/2018 2 DISPOSSIBLES A1 1 15.00 Sub Total 75.00 ICU CHARGES 1 12/05/2018 ICU CHARGES Bed# ICU 1 (34 Hours) 2700.00 Sub Total 2700.00 LABORATORY 1 11/05/2018 19 APTT 1 200.00 2 11/05/2018 19 POE 1 1860.00 3 16/05/2018 SVN190 GRBS 1 40.00 4 16/05/2018 SVN192 BIOPSY L 1 1000.00 5 16/05/2018 SVN192 DRAIN REMOVAL 1 50.00 6 17/05/2018 20 HBAIC 300.00 7 17/05/2018 20 RANDOM BLOOD SUGAR 3 45.00 8 18/05/2018 21 PCV 1 30.00 9 18/05/2018 21 HEAMOGLOBIN 1 30.00 10 18/05/2018 21 RANDOM BLOOD SUGAR 1 15.00 Sub Total 3570.00 MEDICINE CHARGES 1 11/05/2018 OPH1343 CEFBACT.S 1.5 2 309.99 2 11/05/2018 OPH1343 IVF DNS 500 ML INJ 2 62.38 3 11/05/2018 OPH1343 CANULA FIXATOR 1 45.00 4 11/05/2018 OPH1343 DISPOVAN 2 ML 1 4.50 5 11/05/2018 OPH1343 ENEMA PRACTO CLISYS 1 45.00 6 11/05/2018 OPH1343 GLOVES DISPO 1 58.00 7 11/05/2018 OPH1343 IV CANULA 18 G POLYMED 1 102.50 8 11/05/2018 OPH1343 IV SET POLYMED 1 80.00 9 11/05/2018 OPH1343 IVF N S 500 ML INJ 1 28.47 10 11/05/2018 OPH1343 SPIRIT SWAB 2 16.00 11 11/05/2018 OPH1343 IVF DEXTROSE 5 % 500 ML INJ 2 62.03 12 11/05/2018 OPH1343 DISPOVAN 10 ML 2 17.00 13 12/05/2018 OPH1401 ORNIDA 100 ML IV 3 149.99
        """
    ]

#     poller = client.begin_analyze_healthcare_entities(documents)
#     result = poller.result()

#     for doc in result:
#         if not doc.is_error:
#             for entity in doc.entities:
#                 if entity.category in ["MedicationName", "DateTime", "TreatmentName"]:
#                     print("Medicine & Lab Test: {}".format(entity.text))
#                     # print("Category: {}".format(entity.category))

# # Call the function
# extract_medication_and_dates(client)


def health_example(client):
    poller = client.begin_analyze_healthcare_entities(documents)
    result = poller.result()

    docs = [doc for doc in result if not doc.is_error]

    for idx, doc in enumerate(docs):
        for entity in doc.entities:
            if entity.category in ["MedicationName", "TreatmentName", "Date","Time"]:
                print("Entity: {}".format(entity.text))
                print("Category: {}".format(entity.category))

health_example(client)


Entity: 18/05/2018
Category: Date
Entity: 13/05/2018
Category: Time
Entity: 13/05/2018
Category: Time
Entity: 13/05/2018
Category: Time
Entity: ASSI
Category: TreatmentName
Entity: 13/05/2018
Category: Time
Entity: 13/05/2018
Category: Time
Entity: 13/05/2018
Category: Time
Entity: 13/05/2018
Category: Date
Entity: 14/05/2018
Category: Date
Entity: 08
Category: Time
Entity: 20 Hours
Category: Time
Entity: 11/05/2018
Category: Date
Entity: WASTE
Category: TreatmentName
Entity: 15 Hours
Category: Time
Entity: 12/05/2018
Category: Date
Entity: BIOMEDICAL WASTE CHARGE
Category: TreatmentName
Entity: 54 Hours
Category: Time
Entity: 12/05/2018
Category: Date
Entity: BIOMEDICAL WASTE
Category: TreatmentName
Entity: 34 Hours
Category: Time
Entity: 20.00
Category: Time
Entity: 13/05/2018
Category: Time
Entity: BIOMEDICAL WASTE
Category: TreatmentName
Entity: 20 Hours
Category: Time
Entity: 15.00
Category: Time
Entity: 14/05/2018
Category: Time
Entity: BIOMEDICAL WASTE CHARGE
Category: Treatment

In [ ]:
from azure.ai.formrecognizer import FormRecognizerClient
from azure.core.credentials import AzureKeyCredential

# Replace with your Form Recognizer endpoint and key
form_recognizer_endpoint = "https://myhocr.cognitiveservices.azure.com/"
form_recognizer_key = "e9d0eb54060a45889832197a30af244f"

# Authenticate Form Recognizer client
def authenticate_form_recognizer_client():
    fr_credential = AzureKeyCredential(form_recognizer_key)
    form_recognizer_client = FormRecognizerClient(
        endpoint=form_recognizer_endpoint,
        credential=fr_credential)
    return form_recognizer_client

# Function to extract text using Form Recognizer and save to a text file
def extract_text_from_pdf_and_save(form_recognizer_client, pdf_path, output_text_file):
    with open(pdf_path, "rb") as pdf_file:
        poller = form_recognizer_client.begin_recognize_content(pdf_file)
        result = poller.result()
        text = ""
        for page in result:
            for line in page.lines:
                text += line.text + " "

    # Save extracted text to a text file
    with open(output_text_file, "w", encoding="utf-8") as text_file:
        text_file.write(text)

# Example usage
form_recognizer_client = authenticate_form_recognizer_client()
pdf_path = "/content/NHI data requirement #2 (ins fraud detection).pdf"
output_text_file = "D:\Insurance\output\text_file.txt"
extract_text_from_pdf_and_save(form_recognizer_client, pdf_path, output_text_file)

In [ ]:
!pip install azure-ai-textanalytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.6/298.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [ ]:
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

# Replace with your Azure Text Analytics endpoint and key
endpoint = "https://eagle-eyes.cognitiveservices.azure.com/"
key = "de399cd9565a44028fbce14826266cc5"

# Authenticate the client using your key and endpoint
def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint,
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()

# Function to extract medication names and dates from healthcare-related text
def extract_medication_and_dates(client):
    documents = [
        """
MEDICA AL CER SUSHRUTHA MEDICAL CENTRE SO POTHENCODE OF D Pothencode, Thiruvananthapuram Ph:04713247745,419937 Mob:9495989937 BOSUSRUTH Patient Charge Details Registration # Bill # Name Customer Bill Date : 18/05/2018 Address Ward : SINGLE ROOM Bed # : A2 SI # Date Bill # Item Name 1 ANAESTHETIST FEE Qty Amount 1 13/05/2018 SUN32 ICU 6000.00 Sub Total 1 OT CHARGE 6000.00 1 13/05/2018 SUN32 ICU 1 10000 00 Sub Total 1 OT CONSUMABLES 10000.00 1 13/05/2018 SUN32 ICU 1 2000.00 Sub Total 2000.00 1 OT PROF.ASSI. CHARGES 1 13/05/2018 SUN32 ICU 1 S000.00 Sub Total 8000.00 ANAESETHESIA ASSISTANT 1 13/05/2018 SUN32 ICU 1 1000.00 Sub Total 1000.00 ASSISTANT FEE 1 13/05/2018 SUN32 ICU 1 3500.00 Sub Total 3500.00 BED CHARGE 1 13/05/2018 To 14/05/2018 08 :: BED CHARGE Bed# ADL POPS (20 Hours) 1 350.00 Sub Total 350.00 BIOMEDICAL WASTE CHARGE 1 11/05/2018 BIOMEDICAL WASTE CHARGE Bed# A2 (15 Hours) 25.00 2 12/05/2018 BIOMEDICAL WASTE CHARGE Bed# A2 (54 Hours) 50.00 3 12/05/2018 BIOMEDICAL WASTE CHARGE Bed# ICU 1 (34 Hours) 20.00 4 13/05/2018 BIOMEDICAL WASTE CHARGE Bed# ADL POP5 (20 Hours) 15.00 5 14/05/2018 BIOMEDICAL WASTE CHARGE Bed# A2 (98 Hours) 125.00 Sub Total 235.00 CARDIAC MONITORING 1 16/05/2018 SVN192 CARDIAC MONITOR (PER HOUR) 3 150.00 Sub Total 150.00 CONSULTATION CHARGE 1 11/05/2018 CONSULTATION CHARGE Bed# A2 (15 Hours) 450.00 Page 1 of 6 Patient Charge Details registration # Bill # Name Bill Date : 18/05/2018 Customer Ward : SINGLE ROOM Address Bed # : A2 SI # Date Bill # Item Name Qty Amount 2 12/05/2018 CONSULTATION CHARGE Bed# ICU 1 (34 Hours) 600.00 3 13/05/2018 CONSULTATION CHARGE Bed# ADL POPS (20 Hours) 450.00 4 14/05/2018 CONSULTATION CHARGE Bed# A2 (98 Hours) 2250.00 Sub Total 3750.00 CONSUMABLES SALES 1 11/05/2018 19 DISPOSSIBLES A1 1 15.00 2 17/05/2018 20 DISPOSSIBLES A1 3 45.00 3 18/05/2018 2 DISPOSSIBLES A1 1 15.00 Sub Total 75.00 ICU CHARGES 1 12/05/2018 ICU CHARGES Bed# ICU 1 (34 Hours) 2700.00 Sub Total 2700.00 LABORATORY 1 11/05/2018 19 APTT 1 200.00 2 11/05/2018 19 POE 1 1860.00 3 16/05/2018 SVN190 GRBS 1 40.00 4 16/05/2018 SVN192 BIOPSY L 1 1000.00 5 16/05/2018 SVN192 DRAIN REMOVAL 1 50.00 6 17/05/2018 20 HBAIC 300.00 7 17/05/2018 20 RANDOM BLOOD SUGAR 3 45.00 8 18/05/2018 21 PCV 1 30.00 9 18/05/2018 21 HEAMOGLOBIN 1 30.00 10 18/05/2018 21 RANDOM BLOOD SUGAR 1 15.00 Sub Total 3570.00 MEDICINE CHARGES 1 11/05/2018 OPH1343 CEFBACT.S 1.5 2 309.99 2 11/05/2018 OPH1343 IVF DNS 500 ML INJ 2 62.38 3 11/05/2018 OPH1343 CANULA FIXATOR 1 45.00 4 11/05/2018 OPH1343 DISPOVAN 2 ML 1 4.50 5 11/05/2018 OPH1343 ENEMA PRACTO CLISYS 1 45.00 6 11/05/2018 OPH1343 GLOVES DISPO 1 58.00 7 11/05/2018 OPH1343 IV CANULA 18 G POLYMED 1 102.50 8 11/05/2018 OPH1343 IV SET POLYMED 1 80.00 9 11/05/2018 OPH1343 IVF N S 500 ML INJ 1 28.47 10 11/05/2018 OPH1343 SPIRIT SWAB 2 16.00 11 11/05/2018 OPH1343 IVF DEXTROSE 5 % 500 ML INJ 2 62.03 12 11/05/2018 OPH1343 DISPOVAN 10 ML 2 17.00 13 12/05/2018 OPH1401 ORNIDA 100 ML IV 3 149.99
        """
    ]

#     poller = client.begin_analyze_healthcare_entities(documents)
#     result = poller.result()

#     for doc in result:
#         if not doc.is_error:
#             for entity in doc.entities:
#                 if entity.category in ["MedicationName", "DateTime", "TreatmentName"]:
#                     print("Medicine & Lab Test: {}".format(entity.text))
#                     # print("Category: {}".format(entity.category))

# # Call the function
# extract_medication_and_dates(client)


# def health_example(client):
#     poller = client.begin_analyze_healthcare_entities(documents)
#     result = poller.result()

#     docs = [doc for doc in result if not doc.is_error]

#     for idx, doc in enumerate(docs):
#         for entity in doc.entities:
#             if entity.category in ["MedicationName", "TreatmentName", "Date","Time"]:
#                 print("Entity: {}".format(entity.text))
#                 print("Category: {}".format(entity.category))

# health_example(client)

def extract_medication_and_dates(client):
    documents = [
        """
MEDICA AL CER SUSHRUTHA MEDICAL CENTRE SO POTHENCODE OF D Pothencode, Thiruvananthapuram Ph:04713247745,419937 Mob:9495989937 BOSUSRUTH Patient Charge Details Registration # Bill # Name Customer Bill Date : 18/05/2018 Address Ward : SINGLE ROOM Bed # : A2 SI # Date Bill # Item Name 1 ANAESTHETIST FEE Qty Amount 1 13/05/2018 SUN32 ICU 6000.00 Sub Total 1 OT CHARGE 6000.00 1 13/05/2018 SUN32 ICU 1 10000 00 Sub Total 1 OT CONSUMABLES 10000.00 1 13/05/2018 SUN32 ICU 1 2000.00 Sub Total 2000.00 1 OT PROF.ASSI. CHARGES 1 13/05/2018 SUN32 ICU 1 S000.00 Sub Total 8000.00 ANAESETHESIA ASSISTANT 1 13/05/2018 SUN32 ICU 1 1000.00 Sub Total 1000.00 ASSISTANT FEE 1 13/05/2018 SUN32 ICU 1 3500.00 Sub Total 3500.00 BED CHARGE 1 13/05/2018 To 14/05/2018 08 :: BED CHARGE Bed# ADL POPS (20 Hours) 1 350.00 Sub Total 350.00 BIOMEDICAL WASTE CHARGE 1 11/05/2018 BIOMEDICAL WASTE CHARGE Bed# A2 (15 Hours) 25.00 2 12/05/2018 BIOMEDICAL WASTE CHARGE Bed# A2 (54 Hours) 50.00 3 12/05/2018 BIOMEDICAL WASTE CHARGE Bed# ICU 1 (34 Hours) 20.00 4 13/05/2018 BIOMEDICAL WASTE CHARGE Bed# ADL POP5 (20 Hours) 15.00 5 14/05/2018 BIOMEDICAL WASTE CHARGE Bed# A2 (98 Hours) 125.00 Sub Total 235.00 CARDIAC MONITORING 1 16/05/2018 SVN192 CARDIAC MONITOR (PER HOUR) 3 150.00 Sub Total 150.00 CONSULTATION CHARGE 1 11/05/2018 CONSULTATION CHARGE Bed# A2 (15 Hours) 450.00 Page 1 of 6 Patient Charge Details registration # Bill # Name Bill Date : 18/05/2018 Customer Ward : SINGLE ROOM Address Bed # : A2 SI # Date Bill # Item Name Qty Amount 2 12/05/2018 CONSULTATION CHARGE Bed# ICU 1 (34 Hours) 600.00 3 13/05/2018 CONSULTATION CHARGE Bed# ADL POPS (20 Hours) 450.00 4 14/05/2018 CONSULTATION CHARGE Bed# A2 (98 Hours) 2250.00 Sub Total 3750.00 CONSUMABLES SALES 1 11/05/2018 19 DISPOSSIBLES A1 1 15.00 2 17/05/2018 20 DISPOSSIBLES A1 3 45.00 3 18/05/2018 2 DISPOSSIBLES A1 1 15.00 Sub Total 75.00 ICU CHARGES 1 12/05/2018 ICU CHARGES Bed# ICU 1 (34 Hours) 2700.00 Sub Total 2700.00 LABORATORY 1 11/05/2018 19 APTT 1 200.00 2 11/05/2018 19 POE 1 1860.00 3 16/05/2018 SVN190 GRBS 1 40.00 4 16/05/2018 SVN192 BIOPSY L 1 1000.00 5 16/05/2018 SVN192 DRAIN REMOVAL 1 50.00 6 17/05/2018 20 HBAIC 300.00 7 17/05/2018 20 RANDOM BLOOD SUGAR 3 45.00 8 18/05/2018 21 PCV 1 30.00 9 18/05/2018 21 HEAMOGLOBIN 1 30.00 10 18/05/2018 21 RANDOM BLOOD SUGAR 1 15.00 Sub Total 3570.00 MEDICINE CHARGES 1 11/05/2018 OPH1343 CEFBACT.S 1.5 2 309.99 2 11/05/2018 OPH1343 IVF DNS 500 ML INJ 2 62.38 3 11/05/2018 OPH1343 CANULA FIXATOR 1 45.00 4 11/05/2018 OPH1343 DISPOVAN 2 ML 1 4.50 5 11/05/2018 OPH1343 ENEMA PRACTO CLISYS 1 45.00 6 11/05/2018 OPH1343 GLOVES DISPO 1 58.00 7 11/05/2018 OPH1343 IV CANULA 18 G POLYMED 1 102.50 8 11/05/2018 OPH1343 IV SET POLYMED 1 80.00 9 11/05/2018 OPH1343 IVF N S 500 ML INJ 1 28.47 10 11/05/2018 OPH1343 SPIRIT SWAB 2 16.00 11 11/05/2018 OPH1343 IVF DEXTROSE 5 % 500 ML INJ 2 62.03 12 11/05/2018 OPH1343 DISPOVAN 10 ML 2 17.00 13 12/05/2018 OPH1401 ORNIDA 100 ML IV 3 149.99
        """
    ]

#     poller = client.begin_analyze_healthcare_entities(documents)
#     result = poller.result()

#     for doc in result:
#         if not doc.is_error:
#             for entity in doc.entities:
#                 if entity.category in ["MedicationName"]:
#                    print("Entity: {}".format(entity.text))
#                    print("Category: {}".format(entity.category))

# # Call the function
# extract_medication_and_dates(client)

    poller = client.begin_analyze_healthcare_entities(documents)
    result = poller.result()

    for doc in result:
        if not doc.is_error:
            for entity in doc.entities:
                if entity.category in ["MedicationName"]:
                    print("Medicine: {}".format(entity.text))
                    # print("Category: {}".format(entity.category))

# Call the function
extract_medication_and_dates(client)

Medicine: RANDOM
Medicine: OPH1343
Medicine: DISPOVAN
Medicine: OPH1343
Medicine: OPH1343
Medicine: OPH1343
Medicine: S
Medicine: DISPOVAN
Medicine: ORNIDA
